In [ ]:
# -*- coding: utf-8
# Repórter Brasil (https://ruralometro2022.reporterbrasil.org.br/)
# Reinaldo Chaves (@paidatocandeira)
# Faz o cruzamento de deputados e doações - para saber deputados que podem estar no Cadastro de Empregadores que tenham submetido trabalhadores a condições análogas à de escravo ou doadores que estão na lista
# Dados - https://www.gov.br/trabalho-e-previdencia/pt-br/composicao/orgaos-especificos/secretaria-de-trabalho/inspecao/areas-de-atuacao/cadastro_de_empregadores.pdf

In [1]:
import pandas as pd
import unidecode

In [2]:
def f(str):
    try:
        return (unidecode.unidecode(str))
    except:
        return (str)

In [3]:
escravo_fev_2022 = pd.read_excel('originais/lista_trabalho_escravo_fev_2022.xlsx', sheet_name = 'Planilha1', dtype = 'str')

In [4]:
escravo_fev_2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78 entries, 0 to 77
Data columns (total 10 columns):
 #   Column                                                   Non-Null Count  Dtype 
---  ------                                                   --------------  ----- 
 0   ID                                                       78 non-null     object
 1   ano_acao_fiscal                                          78 non-null     object
 2   uf                                                       78 non-null     object
 3   empregador                                               78 non-null     object
 4   cnpj_cpf                                                 78 non-null     object
 5   estabelecimento                                          78 non-null     object
 6   trabalhadores_envolvidos                                 78 non-null     object
 7   cnae                                                     78 non-null     object
 8   decisao_administrativa_de_procedencia_irre

In [5]:
escravo_fev_2022.head()

,ID,ano_acao_fiscal,uf,empregador,cnpj_cpf,estabelecimento,trabalhadores_envolvidos,cnae,decisao_administrativa_de_procedencia_irrecorribilidade,inclusao_no_cadastro_de_empregadores
0,1,2019,PB,A I Maciel Mineração,12.985.350/0001-33,"Sítio Olho D'Água da Viração, Zona Rural, Salg...",12,2399-1/99,2019-12-02 00:00:00,2020-10-05 00:00:00
1,2,2019,MA,A. Richard Tavares Lima,08.229.828/0001-28,"Fazenda Thamia, BR-222, nº 20, Km 10, Povoado ...",19,0151-2/01,2019-11-28 00:00:00,2020-04-03 00:00:00
2,3,2018,DF,Acacio Toratti,068.504.158-16,Fazenda Nova Esperança/Nova Corrente/Gralha Az...,87,0134-2/00,2019-10-23 00:00:00,2020-04-03 00:00:00
3,4,2018,BA,Adilson Bona Vieira,088.062.627-50,Fazenda Dois Rios e Dois Rios I - Rodovia BA-2...,39,0134-2/00,2020-02-27 00:00:00,2021-04-05 00:00:00
4,5,2019,MG,Agrícola Minas Norte Ltda,02.387.202/0001-63,"Fazenda Agricola Minas Norte, Rod. BR-365, km ...",46,1199-0/05,2021-01-08 00:00:00,2021-10-05 00:00:00


In [6]:
def limpeza1(linha):
    documento = str(linha["cnpj_cpf"])
    documento = documento.strip()
    documento = documento.replace('.', '')
    documento = documento.replace('-', '')
    documento = documento.replace('/', '')
    return str(documento)

In [7]:
escravo_fev_2022["cnpj_cpf"] = escravo_fev_2022.apply(limpeza1, axis=1)

In [8]:
# Preenche zeros a esquerda nos documentos
def compara1(linha):
    documento = str(linha["cnpj_cpf"])
    
    if len(documento) < 14 and len(documento) >= 12:
        conta = 14 - (len(documento))
        zero = '0'
        zeros = zero * conta
        alterado = zeros + documento
    elif len(documento) == 14:
        alterado = documento
    elif len(documento) < 11:
        conta = 11 - (len(documento))
        zero = '0'
        zeros = zero * conta
        alterado = zeros + documento
    elif len(documento) == 11:
        alterado = documento
    
    return alterado

In [9]:
escravo_fev_2022["cnpj_cpf"] = escravo_fev_2022.apply(compara1, axis=1)

In [10]:
def limpeza2(linha):
    documento = str(linha["cnpj_cpf"])
    if len(documento) > 11:
        return documento[:8]
    else:
        return "documento_cpf"

escravo_fev_2022['cnpj_raiz'] = escravo_fev_2022.apply(limpeza2, axis=1)

In [11]:
escravo_fev_2022[['cnpj_raiz', 'cnpj_cpf']].sample(10)

,cnpj_raiz,cnpj_cpf
30,documento_cpf,03379915963
36,documento_cpf,73763411291
16,28645167,28645167000129
7,documento_cpf,09511113100
35,27343207,27343207000115
31,17696274,17696274000160
12,documento_cpf,48317829200
4,02387202,02387202000163
20,documento_cpf,36335576600
69,documento_cpf,06946100204


In [12]:
escravo_fev_2022_pf = escravo_fev_2022[(escravo_fev_2022['cnpj_raiz'] == 'documento_cpf')]
escravo_fev_2022_pj = escravo_fev_2022[(escravo_fev_2022['cnpj_raiz'] != 'documento_cpf')]

In [13]:
escravo_fev_2022_pf[['cnpj_raiz', 'cnpj_cpf']].sample(10)

,cnpj_raiz,cnpj_cpf
39,documento_cpf,09715911803
53,documento_cpf,87606348120
3,documento_cpf,08806262750
58,documento_cpf,02655047672
36,documento_cpf,73763411291
69,documento_cpf,06946100204
26,documento_cpf,03220391850
2,documento_cpf,06850415816
49,documento_cpf,37634135100
41,documento_cpf,82545340634


In [14]:
escravo_fev_2022_copia = escravo_fev_2022.copy()

In [15]:
escravo_fev_2022_copia.columns

Index(['ID', 'ano_acao_fiscal', 'uf', 'empregador', 'cnpj_cpf',
       'estabelecimento', 'trabalhadores_envolvidos', 'cnae',
       'decisao_administrativa_de_procedencia_irrecorribilidade',
       'inclusao_no_cadastro_de_empregadores', 'cnpj_raiz'],
      dtype='object')

In [16]:
escravo_fev_2022_copia.rename(columns = {'decisao_administrativa_de_procedencia_irrecorribilidade':'data_administrativa'},inplace = True)
escravo_fev_2022_copia.rename(columns = {'inclusao_no_cadastro_de_empregadores':'data_cadastro'},inplace = True)
escravo_fev_2022_copia.rename(columns = {'empregador':'entidade_nome'},inplace = True)
escravo_fev_2022_copia.rename(columns = {'uf':'estado_id'},inplace = True)
escravo_fev_2022_copia.rename(columns = {'estabelecimento':'entidade_descricao'},inplace = True)
escravo_fev_2022_copia.rename(columns = {'cnpj_cpf':'entidade_id'},inplace = True)

In [17]:
escravo_fev_2022_copia = escravo_fev_2022_copia[['entidade_id', 'entidade_nome', 'entidade_descricao', 'estado_id', 'data_administrativa', 'data_cadastro']]

In [33]:
escravo_fev_2022_copia.to_csv('resultados/autos_mte/lista_trabalho_escravo_fev_2022.csv',index=False)

In [34]:
escravo_fev_2022_copia.to_excel('resultados/autos_mte/lista_trabalho_escravo_fev_2022_todas_colunas.xlsx',sheet_name='Sheet1',index=False)

### Doações

In [18]:
kwargs = {'sep': ',', 'dtype': str, 'encoding': 'utf-8'}
doacoes_2018 = pd.read_csv("resultados/doacoes/doacoes_geral_politicos_2018.csv", **kwargs)

In [19]:
def limpeza3(linha):
    documento = str(linha["NR_CPF_CNPJ_DOADOR"])
    if len(documento) > 11:
        return documento[:8]
    else:
        return "documento_cpf"

doacoes_2018['NR_CPF_CNPJ_DOADOR_raiz'] = doacoes_2018.apply(limpeza3, axis=1)

In [20]:
doacoes_2018_pf = doacoes_2018[(doacoes_2018['NR_CPF_CNPJ_DOADOR_raiz'] == 'documento_cpf')]
doacoes_2018_pj = doacoes_2018[(doacoes_2018['NR_CPF_CNPJ_DOADOR_raiz'] != 'documento_cpf')]

In [21]:
doacoes_pj_escravo = pd.merge(doacoes_2018, escravo_fev_2022_pj.drop_duplicates('cnpj_raiz'), left_on='NR_CPF_CNPJ_DOADOR_raiz', right_on='cnpj_raiz', how = 'left')

In [22]:
conta = doacoes_pj_escravo[pd.notnull(doacoes_pj_escravo['cnpj_cpf'])]
conta.shape

(0, 27)

In [23]:
# Pessoa física

In [24]:
doacoes_pf_escravo = pd.merge(doacoes_2018, escravo_fev_2022_pf.drop_duplicates('cnpj_cpf'), left_on='NR_CPF_CNPJ_DOADOR', right_on='cnpj_cpf', how = 'left')

In [25]:
conta = doacoes_pf_escravo[pd.notnull(doacoes_pf_escravo['cnpj_cpf'])]
conta.shape

(0, 27)

### Doações indiretas

In [26]:
doacoes_indiretas_2018 = pd.read_excel('resultados/doacoes/doacoes_todas_indiretas_politicos_2018.xlsx', sheet_name = 'Sheet1', dtype = 'str')

In [27]:
doacoes_indiretas_2018.shape

(2372, 68)

In [28]:
doacoes_indiretas_2018.columns

Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO',
       'NM_TIPO_ELEICAO', 'CD_ELEICAO', 'DS_ELEICAO', 'DT_ELEICAO', 'ST_TURNO',
       'TP_PRESTACAO_CONTAS', 'DT_PRESTACAO_CONTAS', 'SQ_PRESTADOR_CONTAS',
       'SG_UF', 'SG_UE', 'NM_UE', 'NR_CNPJ_PRESTADOR_CONTA', 'CD_CARGO',
       'DS_CARGO', 'SQ_CANDIDATO', 'NR_CANDIDATO', 'NM_CANDIDATO',
       'NR_CPF_CANDIDATO', 'NR_CPF_VICE_CANDIDATO', 'NR_PARTIDO', 'SG_PARTIDO',
       'NM_PARTIDO', 'CD_FONTE_RECEITA', 'DS_FONTE_RECEITA',
       'CD_ORIGEM_RECEITA', 'DS_ORIGEM_RECEITA', 'CD_NATUREZA_RECEITA',
       'DS_NATUREZA_RECEITA', 'CD_ESPECIE_RECEITA', 'DS_ESPECIE_RECEITA',
       'CD_CNAE_DOADOR', 'DS_CNAE_DOADOR', 'NR_CPF_CNPJ_DOADOR', 'NM_DOADOR',
       'NM_DOADOR_RFB', 'CD_ESFERA_PARTIDARIA_DOADOR',
       'DS_ESFERA_PARTIDARIA_DOADOR', 'SG_UF_DOADOR', 'CD_MUNICIPIO_DOADOR',
       'NM_MUNICIPIO_DOADOR', 'SQ_CANDIDATO_DOADOR', 'NR_CANDIDATO_DOADOR',
       'CD_CARGO_CANDIDATO_DOADOR', 'DS_CARGO_CANDIDATO_DOADOR

In [29]:
doacoes_indiretas_2018['NR_CPF_CNPJ_DOADOR_raiz'] = doacoes_indiretas_2018.apply(limpeza3, axis=1)

In [30]:
doacoes_indiretas_2018_pf = doacoes_indiretas_2018[(doacoes_indiretas_2018['NR_CPF_CNPJ_DOADOR_raiz'] == 'documento_cpf')]
doacoes_indiretas_2018_pj = doacoes_indiretas_2018[(doacoes_indiretas_2018['NR_CPF_CNPJ_DOADOR_raiz'] != 'documento_cpf')]

In [31]:
doacoes_pf_escravo_indireta = pd.merge(doacoes_indiretas_2018_pj, escravo_fev_2022_pf.drop_duplicates('cnpj_cpf'), left_on='cpf_doacoaindireta', right_on='cnpj_cpf', how = 'left')

In [32]:
conta = doacoes_pf_escravo_indireta[pd.notnull(doacoes_pf_escravo_indireta['cnpj_cpf'])].copy()
conta.shape

(0, 80)

### Doações originárias

In [33]:
doacoes_originarias_2018 = pd.read_excel('resultados/doacoes/doacoes_geral_originarias_politicos_2018.xlsx', sheet_name = 'Sheet1', dtype = 'str')

In [34]:
doacoes_originarias_2018.shape

(11060, 25)

In [35]:
doacoes_originarias_2018.columns

Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO',
       'NM_TIPO_ELEICAO', 'CD_ELEICAO', 'DS_ELEICAO', 'DT_ELEICAO', 'ST_TURNO',
       'TP_PRESTACAO_CONTAS', 'DT_PRESTACAO_CONTAS', 'SQ_PRESTADOR_CONTAS',
       'SG_UF', 'NR_CPF_CNPJ_DOADOR_ORIGINARIO', 'NM_DOADOR_ORIGINARIO',
       'NM_DOADOR_ORIGINARIO_RFB', 'TP_DOADOR_ORIGINARIO',
       'CD_CNAE_DOADOR_ORIGINARIO', 'DS_CNAE_DOADOR_ORIGINARIO', 'SQ_RECEITA',
       'DT_RECEITA', 'DS_RECEITA', 'VR_RECEITA', 'NM_CANDIDATO',
       'NR_CPF_CANDIDATO'],
      dtype='object')

In [36]:
def limpeza4(linha):
    documento = str(linha["NR_CPF_CNPJ_DOADOR_ORIGINARIO"])
    if len(documento) > 11:
        return documento[:8]
    else:
        return "documento_cpf"
    
doacoes_originarias_2018['NR_CPF_CNPJ_DOADOR_ORIGINARIO_raiz'] = doacoes_originarias_2018.apply(limpeza4, axis=1)

In [37]:
doacoes_originarias_2018_pf = doacoes_originarias_2018[(doacoes_originarias_2018['NR_CPF_CNPJ_DOADOR_ORIGINARIO_raiz'] == 'documento_cpf')]
doacoes_originarias_2018_pj = doacoes_originarias_2018[(doacoes_originarias_2018['NR_CPF_CNPJ_DOADOR_ORIGINARIO_raiz'] != 'documento_cpf')]

In [38]:
doacoes_pf_escravo_originaria = pd.merge(doacoes_originarias_2018_pf, escravo_fev_2022_pf.drop_duplicates('cnpj_cpf'), left_on='NR_CPF_CNPJ_DOADOR_ORIGINARIO', right_on='cnpj_cpf', how = 'left')

In [39]:
conta = doacoes_pf_escravo_originaria[pd.notnull(doacoes_pf_escravo_originaria['cnpj_cpf'])].copy()
conta.shape

(0, 37)

### Candidatos 2018

In [40]:
kwargs = {'sep': ',', 'dtype': str, 'encoding': 'utf-8'}
politicos_2018 = pd.read_csv("resultados/politicos_2018_09abr.csv", **kwargs)

In [41]:
politicos_escravo = pd.merge(escravo_fev_2022_pf.drop_duplicates('cnpj_cpf'), politicos_2018, left_on='cnpj_cpf', right_on='politico_id', how = 'left')

In [42]:
conta = politicos_escravo[pd.notnull(politicos_escravo['politico_id'])].copy()
conta.shape

(0, 20)